In [1]:
from transformers import AutoConfig, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets.load import load_metric

from tqdm import tqdm
import wandb

from utils import RelationExtractionDataset, DataHelper, ConfigParser
from metric import compute_metrics
import os
import random
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('/opt/ml/dataset/train/train.csv')
aug_data = pd.read_csv('/opt/ml/dataset/train/aug_all.csv')

In [3]:
len(train_data)

32470

In [4]:
print(len(aug_data))

19211


In [5]:
a = pd.concat([train_data,aug_data])

In [6]:
a

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
19206,12176,배우 송윤아와 결혼한 뒤 같은 소속사에 몸담았던 설경구는 2013년 8월 씨제스엔터...,"{'word': '송윤아', 'start_idx': 3, 'end_idx': 5, ...","{'word': '설경구', 'start_idx': 27, 'end_idx': 29...",per:spouse,wikipedia
19207,6075,박근혜 대통령은 조선일보가 주최하는 제5회 아시아 콘퍼런스에 2014년 3월 3일 ...,"{'word': '조선일보', 'start_idx': 9, 'end_idx': 12...","{'word': '방상훈', 'start_idx': 52, 'end_idx': 54...",org:top_members/employees,wikipedia
19208,2021,로마 신화에서 넵투누스는 바다의 신으로 그리스 신화의 포세이돈에 대응하는 신이다.,"{'word': '넵투누스', 'start_idx': 8, 'end_idx': 11...","{'word': '포세이돈', 'start_idx': 30, 'end_idx': 3...",no_relation,wikipedia
19209,22300,"원창왕후(元昌王后, 생몰년 미상)는 고려의 추존왕후로 추존왕인 의조(작제건)의 처이...","{'word': '작제건', 'start_idx': 38, 'end_idx': 40...","{'word': '원창왕후', 'start_idx': 0, 'end_idx': 3,...",per:spouse,wikipedia


In [4]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [5]:
td = helper.row_entity_tokenize(train_data,entity_data,tokenizer)

In [12]:
data = RelationExtractionDataset(
            td, labels=train_data['label'])

In [21]:
tokenizer.decode(td['input_ids'][0])

'[CLS] 〈 Something 〉 는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [26]:
from transformers import AutoTokenizer, AutoModelForMaskedLM,pipeline


In [27]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModelForMaskedLM.from_pretrained("klue/roberta-large")

In [28]:
nlp_fill = pipeline('fill-mask', top_k=5, model=model, tokenizer=tokenizer)

In [ ]:
[MASK]

In [36]:
nlp_fill('김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침 [MASK]를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.')

[{'sequence': '김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침 목포 를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.',
  'score': 0.25800952315330505,
  'token': 9312,
  'token_str': '목포'},
 {'sequence': '김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침, 를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.',
  'score': 0.07216645777225494,
  'token': 16,
  'token_str': ','},
 {'sequence': '김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침 목포시 를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.',
  'score': 0.05739225819706917,
  'token': 31441,
  'token_str': '목포시'},
 {'sequence': '김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침 광주 를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.',
  'score': 0.03156328946352005,
  'token': 4104,
  'token_str': '광주'},
 {'sequence': '김종식 목포시장을 비롯한 목포시청 직원 150여명은 2020학년도 대학수학능력시험이 치러진 14일 아침 관내 를 비롯한 7개 시험장을 찾아 수험생을 응원하고 학부모를 격려했다.',
  'score': 0.026638081297278404,
  'token': 11668,
  'token_str': '관내'}]